## It has code for 2 factors
#### 1. Spell Checking - More spelling error means mostly writeen by un-professionals
####    and may be a fake news
#### 2. Geography

In [843]:
import numpy as np
import pandas as pd

In [844]:
columns=['id','label','statement','subject','speaker','speaker_job','state',
        'party_affiliation','barely_true_count','false_Count',
        'half_true_count','mostly_true_count','pants_on_fire_count','venue_speach'];
df_lair=pd.read_csv('train.tsv',sep='\t',header=None,names=columns,index_col=False);
df_lair.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [845]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Text Proprocessing

#### Function for text preprocessing
###### 1.lowercase the text
###### 2.word tokenization
###### 3.remove stop words and non alphanumeric charaters
###### 4.stemming

In [846]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
def text_preprocessing(df_base):
    df=df_base.copy()
    # lowercase the text
    df['statement']=df['statement'].str.lower()
    # word tokenization
    df['statement']=df['statement'].map(lambda x: nltk.word_tokenize(x))
    # remove stop words and non alphanumeric charaters
    df['statement']=df['statement'].map(lambda x: [w for w in x if (not w in stop_words) and w.isalpha()])
    # lemmatization
    #wordnet_lemmatizer = WordNetLemmatizer()
    #df_train['statement']=df_train['statement'].map(lambda x: wordnet_lemmatizer.lemmatize(x))    
    # stemming
    porter = PorterStemmer()
    df['statement']=df['statement'].map(lambda x: [porter.stem(w) for w in x] )
    return df

#### Calling text_preprocessing function

In [847]:
df_train=text_preprocessing(df_lair)
df_train.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,"[say, anni, list, polit, group, support, abort...",abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,"[declin, coal, start, start, natur, ga, took, ...","energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"[hillari, clinton, agre, john, mccain, vote, g...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,"[health, care, reform, legisl, like, mandat, f...",health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,"[econom, turnaround, start, end, term]","economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


## Spelling Error Check Factor

###### function spell_check_article - It takes an article as input and returns number of words with spelling error in an article and % of words with errors

In [848]:
from nltk.corpus import wordnet
import enchant
enchant = enchant.Dict("en_US")
def spell_check_article(article):
    article=article.replace('-',' ');
    article_tokenized=nltk.word_tokenize(article)
    total_word_count=len(article_tokenized)
    words_with_error_count=0
    for w in article_tokenized:
        if (not wordnet.synsets(w)) and (not enchant.check(w)):
            words_with_error_count+=1
    return words_with_error_count,(words_with_error_count/total_word_count*100)

In [849]:
spell_check_article('kitten drining choclate malk')

(3, 75.0)

In [850]:
spell_check_article('kitten drining choclate milk')

(2, 50.0)

In [851]:
df_lair['spell_error']=df_lair['statement'].map(lambda x: spell_check_article(x)[0])
df_lair[['statement','spell_error']].head()

,statement,spell_error
0,Says the Annies List political group supports ...,1
1,When did the decline of coal start? It started...,3
2,"Hillary Clinton agrees with John McCain ""by vo...",2
3,Health care reform legislation is likely to ma...,0
4,The economic turnaround started at the end of ...,0


## Word2Vec to create a vector representation of processed words for every article

In [852]:
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [853]:
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [854]:
def labelled_sentences(articles, label_type):
    labelledSentences = []
    for i,_d in enumerate(articles):
        labelledSentences.append(LabeledSentence(_d, label_type[i]))
    return labelledSentences

In [855]:
df_train_labelled=labelled_sentences(df_train['statement'],df_train['label'])
df_train_labelled[1]

LabeledSentence(words=['declin', 'coal', 'start', 'start', 'natur', 'ga', 'took', 'start', 'begin', 'presid', 'georg', 'bush', 'administr'], tags='half-true')

In [856]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_train['statement'])]

In [857]:
tagged_data_words=[x.words for x in tagged_data]
tagged_data_words_1D=[]
for row in range(len(tagged_data_words)):
    for col in range(len(tagged_data_words[row])):
        tagged_data_words_1D.append(tagged_data_words[row][col]) 

In [858]:
n_dim=500
w2v = Word2Vec(size=n_dim, min_count=0)
w2v.build_vocab(tagged_data_words)
w2v.train(tagged_data_words,total_examples=model.corpus_count,epochs=model.epochs)

(476194, 524220)

In [859]:
w2v.most_similar('polit')

[('report', 0.9999780058860779),
 ('washington', 0.9999773502349854),
 ('made', 0.9999759197235107),
 ('militari', 0.9999758005142212),
 ('terrorist', 0.9999754428863525),
 ('think', 0.9999743700027466),
 ('union', 0.9999741315841675),
 ('know', 0.9999733567237854),
 ('mayor', 0.9999729990959167),
 ('offici', 0.9999724626541138)]

In [860]:
df_train_statements=df_train[['statement','label']]
df_train_statements_vectorized=df_train_statements['statement'].map(lambda x: [w2v[w] for w in x])

In [861]:
df_train_statements['statement']=df_train_statements_vectorized

In [862]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_train_statements['label']=le.fit_transform(df_train_statements['label'])

In [863]:
df_train_statements.head()

,statement,label
0,"[[-0.072599426, -0.21651855, -0.16844359, -0.3...",1
1,"[[-0.009672652, -0.03996294, -0.036324468, -0....",2
2,"[[-0.056719594, -0.15282656, -0.10569548, -0.2...",3
3,"[[-0.07339665, -0.123614766, -0.1795824, -0.30...",1
4,"[[-0.03877244, -0.13914014, -0.12532502, -0.24...",2


In [864]:
#list(le.inverse_transform(df_train_statements['label']))

In [865]:
def document_vector(doc):
    return np.mean(doc, axis=0)

In [866]:
df_train_statements['statement']=df_train_statements['statement'].apply(document_vector)
df_train_statements.drop(index=4497,inplace=True)
df_train_statements.reset_index(drop=True)

,statement,label
0,"[-0.033912055, -0.11502558, -0.09999106, -0.20...",1
1,"[-0.032747686, -0.11197769, -0.09475077, -0.19...",2
2,"[-0.04219675, -0.13342164, -0.1132291, -0.2327...",3
3,"[-0.041678987, -0.122321546, -0.122972034, -0....",1
4,"[-0.029819334, -0.10638155, -0.0959499, -0.185...",2
5,"[-0.024687247, -0.097756945, -0.09121875, -0.1...",5
6,"[-0.027807025, -0.1080783, -0.10012022, -0.187...",0
7,"[-0.026685696, -0.0971061, -0.08965747, -0.169...",2
8,"[-0.02358354, -0.084220596, -0.07664968, -0.14...",2
9,"[-0.028090931, -0.09806442, -0.08965611, -0.17...",3


In [867]:
arr1=[]
for x in df_train_statements['statement'].values:
    arr1.append(x.tolist())   
X=pd.DataFrame(arr1)
y=df_train_statements[['label']]
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [868]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
3474,-0.028776,-0.091908,-0.082594,-0.167292,-0.060526,-0.130846,0.126696,-0.087382,-0.005830,-0.161243,...,0.028846,-0.233256,0.100577,-0.033023,0.055945,-0.101941,-0.062278,-0.196080,-0.003675,-0.147642
6202,-0.032597,-0.119006,-0.107817,-0.208972,-0.068970,-0.155759,0.163707,-0.109914,-0.013139,-0.199626,...,0.031108,-0.292670,0.117648,-0.044829,0.073813,-0.122350,-0.079317,-0.235775,-0.012632,-0.186685
5868,-0.024261,-0.080747,-0.068491,-0.140656,-0.051851,-0.110521,0.109203,-0.074340,-0.003937,-0.136290,...,0.024675,-0.192575,0.083282,-0.028417,0.049702,-0.085455,-0.053800,-0.164077,-0.000498,-0.123600
7357,-0.042197,-0.155574,-0.139883,-0.277288,-0.096813,-0.209959,0.213628,-0.144027,-0.013384,-0.267636,...,0.043178,-0.382214,0.159358,-0.061552,0.097461,-0.162866,-0.105621,-0.317108,-0.009299,-0.245909
9514,-0.031618,-0.106343,-0.095700,-0.188939,-0.067450,-0.143407,0.145358,-0.099120,-0.011532,-0.183695,...,0.030806,-0.261945,0.111364,-0.040532,0.065856,-0.112845,-0.069738,-0.218501,-0.007068,-0.167584


In [869]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
lr_pred = logisticRegr.predict(X_test)

In [870]:
from sklearn import metrics
# Lets print the confusion matrix
print(metrics.confusion_matrix(y_test,lr_pred))

[[  0 186 200  66   0   0]
 [  0 247 268  98   0   0]
 [  0 216 290 128   1   0]
 [  0 184 255 165   0   0]
 [  0 132  99  23   2   0]
 [  0 179 216 117   0   0]]


In [871]:
print(metrics.classification_report(y_test,lr_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       452
          1       0.22      0.40      0.28       613
          2       0.22      0.46      0.30       635
          3       0.28      0.27      0.27       604
          4       0.67      0.01      0.02       256
          5       0.00      0.00      0.00       512

avg / total       0.20      0.23      0.17      3072



In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Lets try with 100 trees
num_trees=100
rf=RandomForestClassifier(n_estimators=num_trees)
rf.fit(X_train, y_train)
rf_pred=rf.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test,lr_pred))

In [ ]:
print(metrics.classification_report(y_test,lr_pred))